In [1]:
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers import modes, Cipher
from cryptography.hazmat.backends import default_backend
from os import urandom
import sys

In [2]:
print(sys.byteorder)

def xor(x, y):
    assert len(x) == len(y)
    a = int.from_bytes(x, sys.byteorder)
    b = int.from_bytes(y, sys.byteorder)
    r = a ^ b
    return r.to_bytes(len(x), sys.byteorder)

little


In [3]:
x = bytes.fromhex("414243FF41434500") #ABC/xffACE/x00
y = bytes.fromhex("0000000000000000")

print(x)
print(xor(x, x).hex())
print(xor(x, y).hex())
print(xor(y, x).hex())
print(x.hex())

b'ABC\xffACE\x00'
0000000000000000
414243ff41434500
414243ff41434500
414243ff41434500


In [4]:
def unpad(m):
    pad = m[-1]
    
    for i in range(1, pad + 1):
        assert m[-i] == pad, "padding of message is wrong"
    
    return m[:-pad].decode("utf-8")


def pad(pt, b):
    pad_size = len(pt) % b
    if pad_size != 0:
        pad = [pad_size for i in range(pad_size)]
        pt += bytes(pad) 
    else:
        pad = [b for i in range(b)]
        pt += bytes(pad)


def number_of_blocks(len_ct, b): # returns number of blocks
    assert len_ct % b == 0, "length of ciphertext must be a multiple number of bytes per block"
    return len_ct // b


def decrypt_cbc(ct_string, k_string, blocksize=16):
    
    ct = bytes.fromhex(ct_string)
    k = bytes.fromhex(k_string)
    n = number_of_blocks(len(ct), blocksize)
    
    cipher = Cipher(AES(k), modes.ECB(), backend=default_backend())
    aes_decrypt = cipher.decryptor().update
    m = bytearray()
    
    for i in range(n-1):
        start, mid, end = i*blocksize, (i+1)*blocksize, (i+2)*blocksize 
        cx, cy = ct[start:mid], ct[mid:end]
        
        d = aes_decrypt(cy)
        m += xor(cx, d)
    
    return unpad(m)


def encrypt_cbc(pt_string, k_string, blocksize=16):
    
    pt = bytearray(pt_string, 'utf-8')
    k = bytes.fromhex(k_string)
    pad(pt, blocksize)
    current = urandom(blocksize) #IV
    n = number_of_blocks(len(pt), blocksize)
    
    cipher = Cipher(AES(k), modes.ECB(), backend=default_backend())
    aes_encrypt = cipher.encryptor().update
    ct = bytearray(current)
    
    
    for i in range(n):
        start, end = i*blocksize, (i+1)*blocksize
        m = pt[start:end]
        d = xor(m, current)
        current = aes_encrypt(d)
        ct += current
    
    return ct.hex()    

In [5]:
k = "140b41b22a29beb4061bda66b6747e14"

ct1 = "4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81"
ct2 = "5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253"

pt1 = "Basic CBC mode encryption needs padding."
pt2 = "Our implementation uses rand. IV"

m1 = decrypt_cbc(ct1, k)
m2 = decrypt_cbc(ct2, k)

print(m1)
print(m2)

c1 = encrypt_cbc(pt1, k)
c2 = encrypt_cbc(pt2, k)

cx = encrypt_cbc(m1, k)
cy = encrypt_cbc(m2, k)

print(decrypt_cbc(c1, k))
print(decrypt_cbc(cy, k))

assert m1==pt1
assert m2==pt2
assert decrypt_cbc(c1, k)==decrypt_cbc(cx, k)
assert decrypt_cbc(c2, k)==decrypt_cbc(cy, k)

Basic CBC mode encryption needs padding.
Our implementation uses rand. IV
Basic CBC mode encryption needs padding.
Our implementation uses rand. IV
